In [1]:
!pip install qdrant-client>=1.1.1
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 27.7 MB/s eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 15.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 17.4 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information 

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/home/toonies/anaconda3/envs/Qqdrant/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# Let's make a semantic search for Sci-Fi books! 
documents = [
  { "name": "The Time Machine", "description": "A man travels through time and witnesses the evolution of humanity.", "author": "H.G. Wells", "year": 1895 },
  { "name": "Ender's Game", "description": "A young boy is trained to become a military leader in a war against an alien race.", "author": "Orson Scott Card", "year": 1985 },
  { "name": "Brave New World", "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.", "author": "Aldous Huxley", "year": 1932 },
  { "name": "The Hitchhiker's Guide to the Galaxy", "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.", "author": "Douglas Adams", "year": 1979 },
  { "name": "Dune", "description": "A desert planet is the site of political intrigue and power struggles.", "author": "Frank Herbert", "year": 1965 },
  { "name": "Foundation", "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.", "author": "Isaac Asimov", "year": 1951 },
  { "name": "Snow Crash", "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.", "author": "Neal Stephenson", "year": 1992 },
  { "name": "Neuromancer", "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.", "author": "William Gibson", "year": 1984 },
  { "name": "The War of the Worlds", "description": "A Martian invasion of Earth throws humanity into chaos.", "author": "H.G. Wells", "year": 1898 },
  { "name": "The Hunger Games", "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.", "author": "Suzanne Collins", "year": 2008 },
  { "name": "The Andromeda Strain", "description": "A deadly virus from outer space threatens to wipe out humanity.", "author": "Michael Crichton", "year": 1969 },
  { "name": "The Left Hand of Darkness", "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.", "author": "Ursula K. Le Guin", "year": 1969 },
  { "name": "The Time Traveler's Wife", "description": "A love story between a man who involuntarily time travels and the woman he loves.", "author": "Audrey Niffenegger", "year": 2003 }
]

In [5]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [6]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [7]:
# Let's vectorize descriptions and upload to qdrant

qdrant.upload_records(
    collection_name="my_books",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(documents)
    ]
)

In [8]:
# Let's now search for something

hits = qdrant.search(
    collection_name="my_books",
    query_vector=encoder.encode("Aliens attack our planet").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'The War of the Worlds', 'description': 'A Martian invasion of Earth throws humanity into chaos.', 'author': 'H.G. Wells', 'year': 1898} score: 0.5265540495038633
{'name': 'The Andromeda Strain', 'description': 'A deadly virus from outer space threatens to wipe out humanity.', 'author': 'Michael Crichton', 'year': 1969} score: 0.4260535272664947
{'name': "The Hitchhiker's Guide to the Galaxy", 'description': 'A comedic science fiction series following the misadventures of an unwitting human and his alien friend.', 'author': 'Douglas Adams', 'year': 1979} score: 0.36173440995681116


In [9]:
# Let's now search only for books from 21st century

hits = qdrant.search(
    collection_name="my_books",
    query_vector=encoder.encode("Tyranic society").tolist(),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="year",
                range=models.Range(
                    gte=2000
                )
            )
        ]
    ),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'The Hunger Games', 'description': 'A dystopian society where teenagers are forced to fight to the death in a televised spectacle.', 'author': 'Suzanne Collins', 'year': 2008} score: 0.32551047537293154
{'name': "The Time Traveler's Wife", 'description': 'A love story between a man who involuntarily time travels and the woman he loves.', 'author': 'Audrey Niffenegger', 'year': 2003} score: 0.13160474631551444


In [11]:
from qdrant_client import QdrantClient
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np

qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance
qdrant.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size= 512, # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [135]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client = QdrantClient(":memory:")

client.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(size=512, distance=models.Distance.COSINE),
    optimizers_config=models.OptimizersConfigDiff(memmap_threshold=2000),
    quantization_config=models.ProductQuantization(
        product=models.ProductQuantizationConfig(
            compression=models.CompressionRatio.X64,
            always_ram=True,
        ),
    ),
)

True

In [136]:
import numpy as np
from qdrant_client.models import PointStruct

vectors = np.random.rand(5000, 512)
client.upsert(
    collection_name="my_books",
    points=[
        PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 100}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [137]:
vectors[1].size

512

In [148]:
query_vector = np.random.rand(512)

import time
start = time.time()
hits = client.search(
    collection_name="my_books",
    query_vector=query_vector,
    limit=10  # Return 5 closest points
)
end = time.time()
end-start

0.017362356185913086

In [1]:
!pip install autofaiss

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from autofaiss import build_index
import numpy as np

embeddings = np.float32(np.random.rand(2100, 512))
index, index_infos = build_index(embeddings, save_on_disk=False)

2023-08-13 09:20:06,697 [INFO]: Using 8 omp threads (processes), consider increasing --nb_cores if you have more
2023-08-13 09:20:06,701 [INFO]: Launching the whole pipeline 08/13/2023, 09:20:06
2023-08-13 09:20:06,702 [INFO]: Reading total number of vectors and dimension 08/13/2023, 09:20:06
100%|██████████| 1/1 [00:00<00:00, 12409.18it/s]
2023-08-13 09:20:06,795 [INFO]: There are 2100 embeddings of dim 512
2023-08-13 09:20:06,796 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0935 secs
2023-08-13 09:20:06,797 [INFO]: 	Compute estimated construction time of the index 08/13/2023, 09:20:06
2023-08-13 09:20:06,798 [INFO]: 		-> Train: 16.7 minutes
2023-08-13 09:20:06,799 [INFO]: 		-> Add: 0.0 seconds
2023-08-13 09:20:06,800 [INFO]: 		Total: 16.7 minutes
2023-08-13 09:20:06,802 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0032 secs
2023-08-13 09:20:06,802 [INFO]: 	Checking that your have enough memory available to create the index

In [11]:
import time
start = time.time()

query = np.float32(np.random.rand(1, 512))
_, I = index.search(query, 1)

print(time.time() - start)

0.011642694473266602


In [6]:
embeddings.flatten().size

1075200